In [67]:
import csv
import json
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
import joblib
from scipy.stats import binom
import math
import numpy as np
import matplotlib.pyplot as plt
import random
import hashlib




In [78]:
print(os.getcwd())


/home/nev/Documents/Bachelor/RetrospectiveDetection/src


In [82]:
f = open("data/concat_set.jsonl", encoding="utf-8")
df = pd.read_json("data/concat_set.jsonl")

In [93]:
df["tokens_changed"].value_counts()

tokens_changed
0     281
64    160
31      3
26      2
55      2
58      2
41      2
39      2
59      1
37      1
50      1
22      1
23      1
61      1
32      1
60      1
48      1
36      1
57      1
28      1
38      1
30      1
33      1
35      1
Name: count, dtype: int64

In [59]:
# 2) Label robust erstellen (True/False oder "true"/"false")
if df["is_watermarked_true"].dtype == bool:
    df["label"] = df["is_watermarked_true"].astype(int)
else:
    df["label"] = (
        df["is_watermarked_true"]
        .astype(str).str.strip().str.lower()
        .map({"true": 1, "false": 0})
    )

# 3) benötigte Numerik-Spalten sicher in float konvertieren
num_cols = ["green_hits", "tokens_checked", "z_score", "p_value", "tokens_length"]
for c in num_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# 4) Features ableiten
df["hit_rate"] = df["green_hits"] / df["tokens_checked"].clip(lower=1)

feature_cols = ["z_score", "p_value", "hit_rate", "tokens_length", "tokens_checked"]

# 5) Zeilen mit fehlenden Werten verwerfen (nur was wir brauchen)
df = df.dropna(subset=feature_cols + ["label"]).reset_index(drop=True)

X = df[feature_cols]
y = df["label"].astype(int)

# 6) Split + Pipeline (Skalierung + LogReg)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

clf = make_pipeline(
    StandardScaler(),
    LogisticRegression(max_iter=2000, class_weight="balanced")  # balanced hilft bei Imbalance
)

clf.fit(X_train, y_train)

# 7) Evaluation
y_pred = clf.predict(X_test)
y_prob = clf.predict_proba(X_test)[:, 1]

print("Confusion matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nReport:")
print(classification_report(y_test, y_pred, digits=3))
print("ROC-AUC:", roc_auc_score(y_test, y_prob))

Confusion matrix:
[[52  4]
 [ 2 36]]

Report:
              precision    recall  f1-score   support

           0      0.963     0.929     0.945        56
           1      0.900     0.947     0.923        38

    accuracy                          0.936        94
   macro avg      0.931     0.938     0.934        94
weighted avg      0.938     0.936     0.936        94

ROC-AUC: 0.9769736842105263


In [61]:
seed = 42

# 